In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy.stats as stats
import warnings
import math
from colorama import Fore
warnings.filterwarnings("ignore")

from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import statsmodels.api as sm

In [ ]:
loc1=pd.read_csv('/kaggle/input/wind-power-generation-data-forecasting/Location1.csv')
loc2=pd.read_csv('/kaggle/input/wind-power-generation-data-forecasting/Location2.csv')
loc3=pd.read_csv('/kaggle/input/wind-power-generation-data-forecasting/Location3.csv')
loc4=pd.read_csv('/kaggle/input/wind-power-generation-data-forecasting/Location4.csv')

In [3]:
loc1.head()

NameError: name 'loc1' is not defined

In [4]:
loc2.head()

NameError: name 'loc2' is not defined

In [ ]:
loc3.head()

In [ ]:
loc4.head()

In [ ]:
loc1.describe()


In [ ]:
loc2.describe()

In [ ]:
loc3.describe()

In [ ]:
loc4.describe()

In [ ]:
loc1.info()

In [ ]:
loc2.info()

In [ ]:
loc1.isnull().sum()


In [ ]:
loc2.isnull().sum()


In [ ]:
loc3.isnull().sum()


In [ ]:
loc4.isnull().sum()

In [ ]:
loc1['Time']=pd.to_datetime(loc1['Time'])
loc1['Year']=loc1['Time'].dt.year
loc1['Month']=loc1['Time'].dt.month
loc1['Quarter']=loc1['Time'].dt.quarter
loc1['Day']=loc1['Time'].dt.dayofweek

In [ ]:
loc2['Time']=pd.to_datetime(loc2['Time'])
loc2['Year']=loc2['Time'].dt.year
loc2['Month']=loc2['Time'].dt.month
loc2['Quarter']=loc2['Time'].dt.quarter
loc2['Day']=loc2['Time'].dt.dayofweek


In [ ]:
loc3['Time']=pd.to_datetime(loc3['Time'])
loc3['Year']=loc3['Time'].dt.year
loc3['Month']=loc3['Time'].dt.month
loc3['Quarter']=loc3['Time'].dt.quarter
loc3['Day']=loc3['Time'].dt.dayofweek


In [ ]:
loc4['Time']=pd.to_datetime(loc4['Time'])
loc4['Year']=loc4['Time'].dt.year
loc4['Month']=loc4['Time'].dt.month
loc4['Quarter']=loc4['Time'].dt.quarter
loc4['Day']=loc4['Time'].dt.dayofweek


In [ ]:
loc1.head()

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
# Extract the 'Power' column from each dataset
loc1_power = loc1['Power']
loc2_power = loc2['Power']
loc3_power = loc3['Power']
loc4_power = loc4['Power']

# Perform One-Way ANOVA
anova_result = stats.f_oneway(loc1_power, loc2_power, loc3_power, loc4_power)

# Display ANOVA results
print(f"ANOVA F-statistic: {anova_result.statistic:.4f}, p-value: {anova_result.pvalue:.4f}")

# Check if the result is significant (p-value < 0.05)
if anova_result.pvalue < 0.05:
    print("Significant difference found! Performing Tukey's HSD test...\n")
    
    # Create a combined dataframe for Tukey’s test
    df_tukey = pd.DataFrame({
        'Power': list(loc1_power) + list(loc2_power) + list(loc3_power) + list(loc4_power),
        'Location': (['LOC1'] * len(loc1_power)) + 
                    (['LOC2'] * len(loc2_power)) + 
                    (['LOC3'] * len(loc3_power)) + 
                    (['LOC4'] * len(loc4_power))
    })

    # Perform Tukey's HSD test
    tukey = pairwise_tukeyhsd(df_tukey['Power'], df_tukey['Location'], alpha=0.05)

    # Print Tukey test summary
    print(tukey)

else:
    print("No significant difference found between locations.")


In [ ]:
file_paths = {
    "loc1": "/kaggle/input/wind-power-generation-data-forecasting/Location1.csv",
    "loc2": "/kaggle/input/wind-power-generation-data-forecasting/Location2.csv",
    "loc3": "/kaggle/input/wind-power-generation-data-forecasting/Location3.csv",
    "loc4": "/kaggle/input/wind-power-generation-data-forecasting/Location4.csv"
}

# Load data from CSV files
data_frames = {}
for loc, path in file_paths.items():
    df = pd.read_csv(path)  # Read the CSV file
    df['Time'] = pd.to_datetime(df['Time'])  # Convert 'Time' column to datetime
    df['Location'] = loc  # Add location identifier
    data_frames[loc] = df

# Combine all locations into a single DataFrame
data = pd.concat(data_frames.values())

# Extract year and month for analysis
data['Year'] = data['Time'].dt.year
data['Month'] = data['Time'].dt.to_period('M')

# Compute monthly mean Power and Wind Speed for each location
monthly_means = data.groupby(['Month', 'Location'])[['Power', 'windspeed_100m']].mean().reset_index()

# Convert 'Month' back to datetime for plotting
monthly_means['Month'] = monthly_means['Month'].astype(str)
monthly_means['Month'] = pd.to_datetime(monthly_means['Month'])

# Compute annual mean Power and Wind Speed for each location
annual_means = data.groupby(['Year', 'Location'])[['Power', 'windspeed_100m']].mean().reset_index()

### === PLOTS === ###

# 1️⃣ Monthly Mean Power Comparison
plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_means, x='Month', y='Power', hue='Location', marker='o')
plt.xlabel("Time (Year-Month)")
plt.ylabel("Mean Power")
plt.title("Monthly Mean Power Comparison Across Locations")
plt.xticks(rotation=45)
plt.legend(title="Location")
plt.grid(True)
plt.show()

# 2️⃣ Monthly Mean Wind Speed Comparison
plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_means, x='Month', y='windspeed_100m', hue='Location', marker='o')
plt.xlabel("Time (Year-Month)")
plt.ylabel("Mean Wind Speed (100m)")
plt.title("Monthly Mean Wind Speed Comparison Across Locations")
plt.xticks(rotation=45)
plt.legend(title="Location")
plt.grid(True)
plt.show()

# 3️⃣ Annual Trend of Power Generation
plt.figure(figsize=(12, 6))
sns.lineplot(data=annual_means, x='Year', y='Power', hue='Location', marker='o')
plt.xlabel("Year")
plt.ylabel("Mean Power")
plt.title("Annual Trend of Power Generation Across Locations")
plt.legend(title="Location")
plt.grid(True)
plt.show()

# 4️⃣ Annual Trend of Wind Speed
plt.figure(figsize=(12, 6))
sns.lineplot(data=annual_means, x='Year', y='windspeed_100m', hue='Location', marker='o')
plt.xlabel("Year")
plt.ylabel("Mean Wind Speed (100m)")
plt.title("Annual Trend of Wind Speed Across Locations")
plt.legend(title="Location")
plt.grid(True)
plt.show()

# 5️⃣ Boxplot of Power by Location
plt.figure(figsize=(10, 6))
sns.boxplot(data=data, x='Location', y='Power')
plt.xlabel("Location")
plt.ylabel("Power")
plt.title("Distribution of Power Generation Across Locations")
plt.grid(True)
plt.show()

# 6️⃣ Boxplot of Wind Speed by Location
plt.figure(figsize=(10, 6))
sns.boxplot(data=data, x='Location', y='windspeed_10m')
plt.xlabel("Location")
plt.ylabel("Wind Speed (10m)")
plt.title("Distribution of Wind Speed Across Locations")
plt.grid(True)
plt.show()

# Compute annual sum of Power for each location
annual_sum_power = data.groupby(['Year', 'Location'])['Power'].sum().reset_index()

# Aggregate wind direction data by binning into 10-degree intervals
wind_direction_bins = list(range(0, 361, 10))  # Bin wind direction in 10-degree intervals
data['winddirection_binned'] = pd.cut(data['winddirection_100m'], bins=wind_direction_bins, include_lowest=True)

# Compute wind direction frequency for binned values
wind_direction_distribution = data.groupby('winddirection_binned').size().reset_index(name='Frequency')

### === ADDITIONAL PLOTS === ###

# 1️⃣ Annual Total Power Generation (Bar Chart)
plt.figure(figsize=(12, 6))
sns.barplot(data=annual_sum_power, x='Year', y='Power', hue='Location')
plt.xlabel("Year")
plt.ylabel("Total Power Generated")
plt.title("Total Power Generation Per Year Across Locations")
plt.legend(title="Location")
plt.grid(True)
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Function to count outliers based on IQR
def count_outliers(df, column, groupby_col):
    outlier_counts = {}
    
    for location in df[groupby_col].unique():
        loc_data = df[df[groupby_col] == location][column]
        Q1 = loc_data.quantile(0.25)
        Q3 = loc_data.quantile(0.75)
        IQR = Q3 - Q1

        # Outlier range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Count outliers
        outlier_count = ((loc_data < lower_bound) | (loc_data > upper_bound)).sum()
        outlier_counts[location] = outlier_count

    return pd.DataFrame(list(outlier_counts.items()), columns=[groupby_col, 'Outlier Count'])

# Count outliers for wind speed across locations
wind_speed_outliers = count_outliers(data, 'windspeed_10m', 'Location')

# ✅ Display outlier counts
print("Wind Speed Outlier Counts:")
print(wind_speed_outliers)

# ------------------------------
# ✅ Outlier Tracking Over Time
# ------------------------------

# Identify outliers for wind speed with respect to time
def identify_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Mark rows as outliers
    df['is_outlier'] = (df[column] < lower_bound) | (df[column] > upper_bound)
    return df

# Apply outlier identification to wind speed data
data = data.groupby('Location', group_keys=False).apply(lambda x: identify_outliers(x, 'windspeed_10m'))

# Count outliers per month
monthly_outlier_counts = data[data['is_outlier']].groupby(['Month', 'Location']).size().reset_index(name='Outlier Count')

# Convert 'Month' to datetime for proper plotting
monthly_outlier_counts['Month'] = monthly_outlier_counts['Month'].astype(str)
monthly_outlier_counts['Month'] = pd.to_datetime(monthly_outlier_counts['Month'])

# ✅ Display monthly outliers over time
print("\nMonthly Wind Speed Outlier Counts:")
print(monthly_outlier_counts)

### === PLOTS === ###

# 1️⃣ Outliers Over Time (Line Plot)
plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_outlier_counts, x='Month', y='Outlier Count', hue='Location', marker='o')
plt.xlabel("Time (Year-Month)")
plt.ylabel("Outlier Count")
plt.title("Monthly Outliers in Wind Speed Across Locations")
plt.xticks(rotation=45)
plt.legend(title="Location")
plt.grid(True)
plt.show()



In [ ]:
#Continuous variables distribution
var_ind = [1,2,3,4,5,6,7,8,9,10]
plot = plt.figure(figsize = (12, 10))
plot.subplots_adjust(hspace = 0.5, wspace = 0.5)
for i in range(1, 10):
    a = plot.add_subplot(5, 2, i)
    a.hist(loc1.iloc[: , var_ind[i - 1]], alpha = 0.7)
    a.title.set_text('Histogram of ' + loc1.columns[var_ind[i - 1]])

In [ ]:
#Correlation analysis
corr_table1 = round(loc1[['Power', 'temperature_2m', 'relativehumidity_2m', 'dewpoint_2m', 'windspeed_10m', 'windspeed_100m', 'winddirection_10m',
       'winddirection_100m', 'windgusts_10m']].corr(method = 'pearson'), 2)
corr_table1

In [ ]:
fig, axes = plt.subplots(figsize=(8,7))
corrdata=sns.heatmap(round(loc1.iloc[:,1:12].corr(),2), cmap='coolwarm', annot=True,linewidths=0.1)
plt.show()

In [ ]:
fig, axes = plt.subplots(4,3, figsize=(25,20))
sns.kdeplot(np.log(loc1['temperature_2m']),ax=axes[0,0]).set(xlabel='', title='Temperature')
sns.kdeplot(np.log(loc1['relativehumidity_2m']),ax=axes[0,1]).set(xlabel='', title='Relative Humidity 2m')
sns.kdeplot(np.log(loc1['dewpoint_2m']),ax=axes[0,2]).set(xlabel='', title='Dewpoint 2m')
sns.kdeplot(np.log(loc1['windspeed_10m']),ax=axes[1,0]).set(xlabel='', title='Windspeed 10m')
sns.kdeplot(np.log(loc1['windspeed_100m']),ax=axes[1,1]).set(xlabel='', title='Windspeed 100m')
sns.kdeplot(np.log(loc1['winddirection_10m']),ax=axes[1,2]).set(xlabel='', title='Winddirection 10m')
sns.kdeplot(np.log(loc1['winddirection_100m']),ax=axes[2,0]).set(xlabel='', title='Winddirection 100m')
sns.kdeplot(np.log(loc1['windgusts_10m']),ax=axes[2,1]).set(xlabel='', title='windgusts 10m')
sns.kdeplot(np.log(loc1['Power']),ax=axes[2,2]).set(xlabel='', title='Power')
sns.kdeplot(np.log(loc1['Year']),ax=axes[3,0]).set(xlabel='', title='Year')
sns.kdeplot(np.log(loc1['Month']),ax=axes[3,1]).set(xlabel='', title='Month')
plt.show()

In [ ]:
loc1.hist(bins=20, figsize=(15, 15))
plt.show()

In [ ]:
fig, axes = plt.subplots(4,2, figsize=(18,15))
sns.lineplot(data=loc1, y= "Power",x="temperature_2m",palette = 'viridis',ax=axes[0,0]).set(xlabel='temperature_2m',ylabel='Power')
sns.lineplot(data=loc1, y= "Power", x= "relativehumidity_2m",palette = 'deep',ax=axes[0,1]).set(xlabel='relativehumidity_2m',ylabel='Power')
sns.lineplot(data=loc1, y= "Power",x= "dewpoint_2m",palette = 'viridis',ax=axes[1,0]).set(xlabel='dewpoint_2m',ylabel='Power')
sns.lineplot(data=loc1, y= "Power", x= "windspeed_10m",palette = 'deep',ax=axes[1,1]).set(xlabel='windspeed_10m',ylabel='Power')
sns.lineplot(data=loc1, y= "Power", x= "windspeed_100m",palette = 'viridis',ax=axes[2,0]).set(xlabel='windspeed_100m',ylabel='Power')
sns.lineplot(data=loc1, y= "Power", x= "winddirection_10m",palette = 'deep',ax=axes[2,1]).set(xlabel='winddirection_10m',ylabel='Power')
sns.lineplot(data=loc1, y= "Power", x= "winddirection_100m",palette = 'viridis',ax=axes[3,0]).set(xlabel='winddirection_100m',ylabel='Power')
sns.lineplot(data=loc1, y= "Power", x= "windgusts_10m",palette = 'deep',ax=axes[3,1]).set(xlabel='windgusts_10m',ylabel='Power')
plt.show()


In [ ]:

fig, axes = plt.subplots(5,2, figsize=(18,16))
loc1.plot('Time','temperature_2m',ax=axes[0,0])
loc1.plot('Time','relativehumidity_2m',ax=axes[0,1])
loc1.plot('Time','dewpoint_2m',ax=axes[1,0])
loc1.plot('Time','windspeed_10m',ax=axes[1,1])
loc1.plot('Time','windspeed_100m',ax=axes[2,0])
loc1.plot('Time','winddirection_10m',ax=axes[2,1])
loc1.plot('Time','winddirection_100m',ax=axes[3,0])
loc1.plot('Time','windgusts_10m',ax=axes[3,1])
loc1.plot('Time','Power',ax=axes[4,0])


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates  # Correct import for date handling

# Convert 'Time' column to datetime format
loc1['Time'] = pd.to_datetime(loc1['Time'])

# Optional: Resampling to reduce points (e.g., taking mean per day)
loc1_resampled = loc1.resample('D', on='Time').mean()

# Create the plot
plt.figure(figsize=(24, 6))
plt.plot(loc1_resampled.index, loc1_resampled['Power'], linestyle='-', color='b', alpha=0.7, label='Power')

# Formatting the plot
plt.xlabel("Time")
plt.ylabel("Power")
plt.title("Power vs Time (5-Year View)")

# Correctly set the X-axis ticks and labels
plt.gca().xaxis.set_major_locator(mdates.YearLocator(1))  # Show 1-year intervals
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Format as 'YYYY'

# Adjustments for readability
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Extract the hour from the time column
data['Hour'] = data['Time'].dt.hour

# Compute the mean power per hour for each location
hourly_mean_power = data.groupby(['Hour', 'Location'])['Power'].mean().reset_index()

### === PLOTS === ###

# 1️⃣ Line Plot: Average Power Generation per Hour
plt.figure(figsize=(12, 6))
sns.lineplot(data=hourly_mean_power, x='Hour', y='Power', hue='Location', marker='o')
plt.xlabel("Hour of the Day")
plt.ylabel("Average Power")
plt.title("Hourly Power Generation Comparison Across Locations")
plt.xticks(range(0, 24))
plt.legend(title="Location")
plt.grid(True)
plt.show()

